In [1]:
#參考今日教材，試著使用經緯度一圈的長度比這一概念，組合出一個新特徵，
#再觀察原特徵加上新特徵是否提升了正確率?
library(magrittr)
library(caret)
library(data.table)
taxi=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\taxi_data1.csv',sep=',')
str(taxi)

##decompose time
year=month=day=hour=min=sec=weekday=week=rep(0,nrow(taxi))
for (i in 1:nrow(taxi)) {
  t=as.character(taxi$pickup_datetime[i]) %>% strsplit(.," ") %>% unlist(.)
  time.1=t[1]
  time.2=t[2]
  
  t1=time.1%>%strsplit(.,"-")%>% unlist(.) %>% as.numeric()
  t2=time.2%>%strsplit(.,":")%>% unlist(.)%>% as.numeric()
  weekday[i]=weekdays(as.Date(time.1))
  week[i]=week(as.Date(time.1))
  
  year[i]=t1[1]
  month[i]=t1[2]
  day[i]=t1[3]
  
  hour[i]=t2[1]
  min[i]=t2[2]
  sec[i]=t2[3]
}

long.lat=function(latitude){
  ratio= (111.320*cos(latitude*pi/180))/110.574
  return(ratio)
}
# pick-up 經緯度長度比
pick.up.ratio=long.lat(taxi$pickup_latitude)

# drop-off 經緯度長度比
drop.off.ratio=long.lat(taxi$dropoff_latitude)

taxi.old=cbind(taxi[,-2],year,month,day,hour,min,sec)
taxi.new=cbind(taxi[,-2],year,month,day,hour,min,sec,pick.up.ratio,drop.off.ratio)



## regression 
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv1= train(fare_amount~., data=taxi.old, trControl=train_control, method="lm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
rmse1=reg.cv1$results[2]
rsq1=reg.cv1$results[3]

##xgbppst 
t11=Sys.time()
xgb1=train(fare_amount~., data=taxi.old, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.req1=mean(xgb1$results$Rsquared)
gb.rmse1=mean(xgb1$results$RMSE)
t12=Sys.time()

time.gb1=difftime(t11,t12,units='secs')




## regression +長度比
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv2= train(fare_amount~., data=taxi.new, trControl=train_control, method="lm")
t12=Sys.time()
time2=difftime(t11,t12,units='secs')
rmse2=reg.cv2$results[2]
rsq2=reg.cv2$results[3]

##xgbppst +長度比
t11=Sys.time()
xgb2=train(fare_amount~., data=taxi.new, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.req2=mean(xgb2$results$Rsquared)
gb.rmse2=mean(xgb2$results$RMSE)
t12=Sys.time()

time.gb2=difftime(t11,t12,units='secs')


result1=data.frame(type=c('reg','reg+ratio','xgb','xgb+ratio'),
                   rsq=as.numeric(c(rsq1,rsq2,gb.req1,gb.req2)),
                   rmse=as.numeric(c(rmse1,rmse2,gb.rmse1,gb.rmse2)),
                   time=-c(time1,time2,time.gb1,time.gb2))
result1

#試著只使用新特徵估計目標值(忽略原特徵)，效果跟作業1的結果比較起來效果如何?
taxi.new.new=taxi.new[,-(2:5)]

## regression +remove origin +長度比
t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
reg.cv3= train(fare_amount~., data=taxi.new.new, trControl=train_control, method="lm")
t12=Sys.time()
time3=difftime(t11,t12,units='secs')
rmse3=reg.cv3$results[2]
rsq3=reg.cv3$results[3]

##xgbppst  +remove origin +長度比
t11=Sys.time()
xgb3=train(fare_amount~., data=taxi.new.new, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.req3=mean(xgb3$results$Rsquared)
gb.rmse3=mean(xgb3$results$RMSE)
t12=Sys.time()

time.gb3=difftime(t11,t12,units='secs')


result2=data.frame(type=c('reg','reg+ratio','reg+ratio+remove','xgb','xgb+ratio','xgb+ratio+remove'),
                   rsq=as.numeric(c(rsq1,rsq2,rsq3,gb.req1,gb.req2,gb.req3)),
                   rmse=as.numeric(c(rmse1,rmse2,rmse3,gb.rmse1,gb.rmse2,gb.rmse3)),
                   time=-c(time1,time2,time3,time.gb1,time.gb2,time.gb3))
result2



Loading required package: lattice
Loading required package: ggplot2


'data.frame':	5000 obs. of  7 variables:
 $ fare_amount      : num  4.5 12 6.5 6.5 11 6.1 6.1 5 4.5 6.9 ...
 $ pickup_datetime  : Factor w/ 4999 levels "2009-01-01 19:58:00 UTC",..: 2161 4716 4061 338 4249 1960 370 27 1216 2106 ...
 $ pickup_longitude : num  -74 -74 -74 -74 -74 ...
 $ pickup_latitude  : num  40.8 40.7 40.7 40.8 40.7 ...
 $ dropoff_longitude: num  -74 -74 -74 -74 -74 ...
 $ dropoff_latitude : num  40.8 40.7 40.7 40.8 40.8 ...
 $ passenger_count  : int  2 1 2 3 3 1 1 1 1 1 ...


type,rsq,rmse,time
reg,0.02757624,9.624155,1.8937328 secs
reg+ratio,0.03600378,9.583878,0.7295811 secs
xgb,0.63262658,5.814612,56.1137080 secs
xgb+ratio,0.64331078,5.787823,56.2907841 secs


type,rsq,rmse,time
reg,0.027576244,9.624155,1.8937328 secs
reg+ratio,0.036003776,9.583878,0.7295811 secs
reg+ratio+remove,0.009601179,9.730439,0.8974838 secs
xgb,0.632626578,5.814612,56.1137080 secs
xgb+ratio,0.643310775,5.787823,56.2907841 secs
xgb+ratio+remove,0.399583071,7.563350,50.2178190 secs


### (1) 對於 regression與xgb來說，增添ratio確實能提升準確度，但幅度不高。
### (2) 移除pick-up 與 drop-off 的經緯度，只保留ratio，這樣的作法反而使regression與xgb表現變差。
### (3) ratio無法有效地囊括經緯度的訊息，從其轉換公式便可略知 (轉換公式只用到"緯度")。